In [1]:
import json
import newspaper
import numpy as np
import pandas as pd
import pickle
import requests
import re
from bs4 import BeautifulSoup
from newspaper import Article
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [2]:
source_code = requests.get('https://www.4imn.com/top200/').text
soup = BeautifulSoup(source_code)
all_td = soup.find_all('td', {'class': 'i'})
links = []
for i in range(0, len(all_td)):
  #print(((str(all_td[i]).split('src=')[1]).split('/>')[0]).split('=')[1].strip('"'))
  links.append(((str(all_td[i]).split('src=')[1]).split('/>')[0]).split('=')[1].strip('"'))


source_code = requests.get('https://blog.feedspot.com/world_news_blogs/').text
soup = BeautifulSoup(source_code)
all_td = soup.find_all('a', {'class': 'ext'})

for i in range(0, len(all_td)):
  #print(((str(all_td[i]).split('src=')[1]).split('/>')[0]).split('=')[1].strip('"'))
  links.append('https://'+((str(all_td[i]).split('target="_blank">')[1]).split('</a>')[0]))


source_code = requests.get('https://blog.feedspot.com/usa_news_websites/#').text
soup = BeautifulSoup(source_code)
all_td = soup.find_all('a', {'class': 'ext'})

for i in range(0, len(all_td)):
  #print(((str(all_td[i]).split('src=')[1]).split('/>')[0]).split('=')[1].strip('"'))
  links.append('https://'+((str(all_td[i]).split('target="_blank">')[1]).split('</a>')[0]))

print("total websites found: {}".format(len(links)))

#links = links[:1]  ##limit for testing purpose
print(links)

total websites found: 200
['http://www.nytimes.com', 'http://www.theguardian.com', 'http://www.washingtonpost.com', 'http://www.dailymail.co.uk', 'http://www.kompas.com', 'http://www.ltn.com.tw', 'http://www.usatoday.com', 'http://www.wsj.com', 'http://www.telegraph.co.uk', 'http://www.chinadaily.com.cn', 'http://www.independent.co.uk', 'http://elpais.com', 'http://www.marca.com', 'http://www.latimes.com', 'http://nypost.com', 'http://www.manoramaonline.com', 'http://www.ft.com', 'http://www.chron.com', 'http://www.repubblica.it', 'http://www.inquirer.net', 'http://www.thesun.co.uk', 'http://www.lemonde.fr', 'http://www.mirror.co.uk', 'http://www.nikkei.com', 'https://www.elbalad.news', 'http://www.express.co.uk', 'http://www.elmundo.es', 'http://as.com', 'http://www.bild.de', 'http://www.asahi.com', 'http://www.lefigaro.fr', 'http://www.kp.ru', 'http://thehill.com', 'http://www.hurriyet.com.tr', 'http://www.chicagotribune.com', 'http://udn.com', 'http://www.welt.de', 'http://www.infob

In [3]:
list(links)

['http://www.nytimes.com',
 'http://www.theguardian.com',
 'http://www.washingtonpost.com',
 'http://www.dailymail.co.uk',
 'http://www.kompas.com',
 'http://www.ltn.com.tw',
 'http://www.usatoday.com',
 'http://www.wsj.com',
 'http://www.telegraph.co.uk',
 'http://www.chinadaily.com.cn',
 'http://www.independent.co.uk',
 'http://elpais.com',
 'http://www.marca.com',
 'http://www.latimes.com',
 'http://nypost.com',
 'http://www.manoramaonline.com',
 'http://www.ft.com',
 'http://www.chron.com',
 'http://www.repubblica.it',
 'http://www.inquirer.net',
 'http://www.thesun.co.uk',
 'http://www.lemonde.fr',
 'http://www.mirror.co.uk',
 'http://www.nikkei.com',
 'https://www.elbalad.news',
 'http://www.express.co.uk',
 'http://www.elmundo.es',
 'http://as.com',
 'http://www.bild.de',
 'http://www.asahi.com',
 'http://www.lefigaro.fr',
 'http://www.kp.ru',
 'http://thehill.com',
 'http://www.hurriyet.com.tr',
 'http://www.chicagotribune.com',
 'http://udn.com',
 'http://www.welt.de',
 'http:

In [4]:
links_dict = dict(zip(np.arange(1,len(links)+1), links))

In [5]:
links_dict

{1: 'http://www.nytimes.com',
 2: 'http://www.theguardian.com',
 3: 'http://www.washingtonpost.com',
 4: 'http://www.dailymail.co.uk',
 5: 'http://www.kompas.com',
 6: 'http://www.ltn.com.tw',
 7: 'http://www.usatoday.com',
 8: 'http://www.wsj.com',
 9: 'http://www.telegraph.co.uk',
 10: 'http://www.chinadaily.com.cn',
 11: 'http://www.independent.co.uk',
 12: 'http://elpais.com',
 13: 'http://www.marca.com',
 14: 'http://www.latimes.com',
 15: 'http://nypost.com',
 16: 'http://www.manoramaonline.com',
 17: 'http://www.ft.com',
 18: 'http://www.chron.com',
 19: 'http://www.repubblica.it',
 20: 'http://www.inquirer.net',
 21: 'http://www.thesun.co.uk',
 22: 'http://www.lemonde.fr',
 23: 'http://www.mirror.co.uk',
 24: 'http://www.nikkei.com',
 25: 'https://www.elbalad.news',
 26: 'http://www.express.co.uk',
 27: 'http://www.elmundo.es',
 28: 'http://as.com',
 29: 'http://www.bild.de',
 30: 'http://www.asahi.com',
 31: 'http://www.lefigaro.fr',
 32: 'http://www.kp.ru',
 33: 'http://thehi

In [6]:
keys=[1,2,3,4,5,6,7,8,9,10,11,14,15,23,35,70,74,75,77,84,95,104,134,186]
[links_dict.get(key) for key in keys]

['http://www.nytimes.com',
 'http://www.theguardian.com',
 'http://www.washingtonpost.com',
 'http://www.dailymail.co.uk',
 'http://www.kompas.com',
 'http://www.ltn.com.tw',
 'http://www.usatoday.com',
 'http://www.wsj.com',
 'http://www.telegraph.co.uk',
 'http://www.chinadaily.com.cn',
 'http://www.independent.co.uk',
 'http://www.latimes.com',
 'http://nypost.com',
 'http://www.mirror.co.uk',
 'http://www.chicagotribune.com',
 'http://www.japantimes.co.jp',
 'http://www.washingtontimes.com',
 'http://www.mercurynews.com',
 'http://www.seattletimes.com',
 'http://www.chinatimes.com',
 'https://appledaily.com',
 'http://www.straitstimes.com',
 'http://www.dailystar.co.uk',
 'http://www.dailytelegraph.com.au']

In [7]:
new_links=[links_dict.get(key) for key in keys]

In [8]:
def get_top_k_predictions(model,X_test,k):
    
    probs = model.predict_proba(X_test)
    best_n = np.argsort(probs, axis=1)[:,-k:]
    preds=[[model.classes_[predicted_cat] for predicted_cat in prediction] for prediction in best_n]
    preds=[ item[::-1] for item in preds]
    
    return preds

In [9]:
analyzer = SentimentIntensityAnalyzer()

LIMIT = 10
data = pd.DataFrame(columns=['website', 'title', 'link', 'date', 'Text','site_name'])
count = 1

for link in new_links:
    print("Building site for ", link)
    paper = newspaper.build(link, memoize_articles=False)
    print("number of articles = ",paper.size())
    newsPaper = {
      "link": link,
      "articles": []
    }
    noneTypeCount = 0

    for content in paper.articles:
        if count > LIMIT:
            break
        try:
            content.download()
            content.parse()
        except Exception as e:
            print(e)
            print("continuing...")
            continue
        if content.publish_date is None:
            count = count + 1
            continue
        site_name=re.sub('^https?:\/\/(www.)?','',str(link))
        site_name=re.sub('[^0-9a-zA-Z]+','_',site_name)
        site_name=re.sub('_$','',site_name)            
        data = data.append({'website': paper.brand, 'title': content.title, 'link': content.url, 
                          'date': content.publish_date.isoformat(), 'Text': content.text,'site_name':site_name
                           }, ignore_index=True)

        
        noneTypeCount = 0

        ind_file_name = "Result_files/"+site_name+ "_preprocess.csv"
        print("\n\t",count,ind_file_name,"\n")
        count = count + 1
        data.to_csv(ind_file_name, mode='a+',index=False,header=None)
        data = {}
        data = pd.DataFrame(columns=['website', 'title', 'link', 'date', 'Text','site_name'])
    count = 1

Building site for  http://www.nytimes.com
number of articles =  104

	 1 Result_files/nytimes_com_preprocess.csv 


	 2 Result_files/nytimes_com_preprocess.csv 


	 3 Result_files/nytimes_com_preprocess.csv 


	 4 Result_files/nytimes_com_preprocess.csv 


	 5 Result_files/nytimes_com_preprocess.csv 


	 6 Result_files/nytimes_com_preprocess.csv 


	 7 Result_files/nytimes_com_preprocess.csv 


	 8 Result_files/nytimes_com_preprocess.csv 


	 9 Result_files/nytimes_com_preprocess.csv 


	 10 Result_files/nytimes_com_preprocess.csv 

Building site for  http://www.theguardian.com
number of articles =  175

	 1 Result_files/theguardian_com_preprocess.csv 


	 2 Result_files/theguardian_com_preprocess.csv 


	 3 Result_files/theguardian_com_preprocess.csv 


	 4 Result_files/theguardian_com_preprocess.csv 


	 5 Result_files/theguardian_com_preprocess.csv 


	 6 Result_files/theguardian_com_preprocess.csv 


	 7 Result_files/theguardian_com_preprocess.csv 


	 8 Result_files/theguardian_co

Article `download()` failed with 403 Client Error: Forbidden for url: https://www.wsj.com/news/business/defense-aerospace?mod=nav_top_subsection on URL https://www.wsj.com/news/business/defense-aerospace?mod=nav_top_subsection
continuing...
Article `download()` failed with 403 Client Error: Forbidden for url: https://www.wsj.com/news/business/transportation?mod=nav_top_subsection on URL https://www.wsj.com/news/business/transportation?mod=nav_top_subsection
continuing...
Article `download()` failed with 403 Client Error: Forbidden for url: https://www.wsj.com/news/markets/real-estate-commercial?mod=nav_top_subsection on URL https://www.wsj.com/news/markets/real-estate-commercial?mod=nav_top_subsection
continuing...
Article `download()` failed with 403 Client Error: Forbidden for url: https://www.wsj.com/news/business/consumer-products?mod=nav_top_subsection on URL https://www.wsj.com/news/business/consumer-products?mod=nav_top_subsection
continuing...
Article `download()` failed with 4

Article `download()` failed with 403 Client Error: Forbidden for url: https://www.wsj.com/news/author/laura-saunders?mod=nav_top_subsection on URL https://www.wsj.com/news/author/laura-saunders?mod=nav_top_subsection
continuing...
Article `download()` failed with 403 Client Error: Forbidden for url: https://www.wsj.com/news/author/james-mackintosh?page=23?mod=nav_top_subsection on URL https://www.wsj.com/news/author/james-mackintosh?page=23?mod=nav_top_subsection
continuing...
Article `download()` failed with 403 Client Error: Forbidden for url: https://www.wsj.com/news/opinion?mod=nav_top_section on URL https://www.wsj.com/news/opinion?mod=nav_top_section
continuing...
Article `download()` failed with 403 Client Error: Forbidden for url: http://www.wsj.com/news/author/7836?mod=nav_top_subsection on URL http://www.wsj.com/news/author/7836?mod=nav_top_subsection
continuing...
Article `download()` failed with 403 Client Error: Forbidden for url: https://www.wsj.com/news/author/sadanand-d

Article `download()` failed with 403 Client Error: Forbidden for url: https://www.wsj.com/news/life-arts/books?mod=nav_top_subsection on URL https://www.wsj.com/news/life-arts/books?mod=nav_top_subsection
continuing...
Article `download()` failed with 403 Client Error: Forbidden for url: https://www.wsj.com/news/life-arts/automotive?mod=nav_top_subsection on URL https://www.wsj.com/news/life-arts/automotive?mod=nav_top_subsection
continuing...
Article `download()` failed with 403 Client Error: Forbidden for url: https://www.wsj.com/news/life-arts/food-cooking-drink?mod=nav_top_subsection on URL https://www.wsj.com/news/life-arts/food-cooking-drink?mod=nav_top_subsection
continuing...
Article `download()` failed with 403 Client Error: Forbidden for url: https://www.wsj.com/news/life-arts/health-wellness?mod=nav_top_subsection on URL https://www.wsj.com/news/life-arts/health-wellness?mod=nav_top_subsection
continuing...
Article `download()` failed with 403 Client Error: Forbidden for url

Article `download()` failed with 403 Client Error: Forbidden for url: https://www.wsj.com/news/archive/?mod=wsjfooter on URL https://www.wsj.com/news/archive/?mod=wsjfooter
continuing...
Building site for  http://www.telegraph.co.uk
number of articles =  922

	 1 Result_files/telegraph_co_uk_preprocess.csv 


	 2 Result_files/telegraph_co_uk_preprocess.csv 


	 3 Result_files/telegraph_co_uk_preprocess.csv 


	 4 Result_files/telegraph_co_uk_preprocess.csv 


	 5 Result_files/telegraph_co_uk_preprocess.csv 


	 6 Result_files/telegraph_co_uk_preprocess.csv 


	 7 Result_files/telegraph_co_uk_preprocess.csv 


	 8 Result_files/telegraph_co_uk_preprocess.csv 


	 9 Result_files/telegraph_co_uk_preprocess.csv 


	 10 Result_files/telegraph_co_uk_preprocess.csv 

Building site for  http://www.chinadaily.com.cn
number of articles =  696

	 1 Result_files/chinadaily_com_cn_preprocess.csv 


	 2 Result_files/chinadaily_com_cn_preprocess.csv 


	 3 Result_files/chinadaily_com_cn_preprocess.csv

Building prefix dict from C:\ProgramData\Anaconda3\lib\site-packages\jieba\dict.txt ...
Dumping model to file cache C:\Users\user\AppData\Local\Temp\jieba.cache
Loading model cost 2.0540499687194824 seconds.
Prefix dict has been built succesfully.



	 1 Result_files/chinatimes_com_preprocess.csv 


	 2 Result_files/chinatimes_com_preprocess.csv 


	 3 Result_files/chinatimes_com_preprocess.csv 


	 4 Result_files/chinatimes_com_preprocess.csv 


	 5 Result_files/chinatimes_com_preprocess.csv 


	 6 Result_files/chinatimes_com_preprocess.csv 


	 7 Result_files/chinatimes_com_preprocess.csv 


	 8 Result_files/chinatimes_com_preprocess.csv 


	 9 Result_files/chinatimes_com_preprocess.csv 


	 10 Result_files/chinatimes_com_preprocess.csv 

Building site for  https://appledaily.com
number of articles =  123

	 10 Result_files/appledaily_com_preprocess.csv 

Building site for  http://www.straitstimes.com
number of articles =  327

	 1 Result_files/straitstimes_com_preprocess.csv 


	 2 Result_files/straitstimes_com_preprocess.csv 


	 3 Result_files/straitstimes_com_preprocess.csv 


	 4 Result_files/straitstimes_com_preprocess.csv 


	 5 Result_files/straitstimes_com_preprocess.csv 


	 6 Result_files/straitstimes_com_preprocess.c